# Pipelines

In [8]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [9]:
# load dataset from seaborn
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Let's Plan

Impute Missing values -> OHE -> Scaling -> feature selection ->Train model using decisiontree

In [10]:
# drop unnecessary columns
df.drop(columns=['sibsp', 'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone'], inplace=True)

In [11]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['survived']), df['survived'], test_size=0.2, random_state=42)

In [12]:
X_train.head()

,pclass,sex,age,parch,fare,embarked
331,1,male,45.5,0,28.5000,S
733,2,male,23.0,0,13.0000,S
382,3,male,32.0,0,7.9250,S
704,3,male,26.0,0,7.8542,S
813,3,female,6.0,2,31.2750,S


In [13]:
y_train.sample(5)

123    1
116    0
189    0
595    0
589    0
Name: survived, dtype: int64

In [14]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [5])
], remainder='passthrough')

In [15]:
# onehot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,5])
],remainder='passthrough')

In [16]:
# scaling
trf3= ColumnTransformer([
    ('Scale', MinMaxScaler(), slice(0,10))
], remainder= 'passthrough')

In [17]:
# feature scaling
trf4 = SelectKBest(score_func=chi2, k=5)

In [18]:
trf5 = DecisionTreeClassifier()

# Create pipeline

In [19]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5',trf5)
])

# Pipeline Vs make_pipeline
pipeline requires naming of steps, make_pipeline does not

(same applies to column transformer vs make_colum_transformer)

In [20]:
# 1️⃣ Numeric + categorical preprocessing in one ColumnTransformer
trf1 = ColumnTransformer([
    # Numeric columns: impute + scale
    ('trf1_num', Pipeline([
        ('imputer', SimpleImputer()),
        ('scaler', MinMaxScaler())
    ]), ['age','fare','pclass','parch']),
    
    # Categorical columns: impute + one-hot encode
    ('trf1_cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ]), ['sex','embarked'])
])

# 2️⃣ Feature selection
trf2 = SelectKBest(score_func=chi2, k=5)

# 3️⃣ Model
trf3 = DecisionTreeClassifier()

# 4️⃣ Full pipeline
pipe = Pipeline([
    ('trf1', trf1),      # preprocessing
    ('trf2', trf2),      # feature selection
    ('trf3', trf3)       # model
])

# 5️⃣ Train
pipe.fit(X_train, y_train)


Pipeline(steps=[('trf1',
                 ColumnTransformer(transformers=[('trf1_num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['age', 'fare', 'pclass',
                                                   'parch']),
                                                 ('trf1_cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex', 'embarked'])])),
                ('trf2',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000001F30126FE20>)),
                ('trf3', DecisionTreeClassifier())])

In [21]:
from sklearn import set_config
set_config(display='diagram')



In [32]:
pipe.named_steps['trf1'].transformers_[1][1]

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('ohe',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [33]:
# Make predictions
y_pred = pipe.predict(X_test)

In [34]:
# Evaluate the model's accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8044692737430168

# Cross Validation using pipeline

In [35]:
# Cross-validation
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv= 5, scoring='accuracy').mean()

np.float64(0.789421845759874)

# Gridsearch using pipeline

In [26]:
params ={
    'trf5__max_depth':[1,2,3,4, None]
}

In [27]:
params = {
    'trf2__k': [3, 5, 7],  # SelectKBest
    'trf3__criterion': ['gini', 'entropy'],  # DecisionTreeClassifier
    'trf3__max_depth': [None, 5, 10]
}


In [36]:
# Hyperparameter tuning with GridSearchCV
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(transformers=[('trf1_num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['age',
                                                                          'fare',
                                                                          'pclass',
                                                                          'parch']),
                                                                        ('trf1_cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['sex',
                                                                          'embarked'])])),
                                       ('trf2',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x000001F30126FE20>)),
                                       ('trf3', DecisionTreeClassifier())]),
             param_grid={'trf2__k': [3, 5, 7],
                         'trf3__criterion': ['gini', 'entropy'],
                         'trf3__max_depth': [None, 5, 10]},
             scoring='accuracy')

In [40]:
# Display the best cross-validated score achieved by the model during GridSearch
grid.best_score_

np.float64(0.8188318723529993)

In [41]:
# Display the parameter combination that gave the best performance during GridSearch
grid.best_params_

{'trf2__k': 7, 'trf3__criterion': 'entropy', 'trf3__max_depth': 10}

# Exploring the pipeline

In [50]:
# Save the trained pipeline object to a file named 'pipe.pkl' using pickle
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))

# Predict using pipeline

In [43]:
# import libraries
import pickle
import numpy as np

In [48]:
# load the pipeline
pipe = pickle.load(open('pipe.pkl' ,'rb'))

In [49]:
test_input2 = np.array([2, 'male', 31.0, 0,0,10.5, 'S'], dtype=object).reshape(1,7)